# Seq2Vec matching
## Overview
This demo shows Seq2Vec matching technology between books and retells:
 - extracting book frame and corresponding retell frame using parallel corpus relations
 - sending frames data to Seq2Vec matcher, which matches book and retell sentences
 - writing output data to separate .json file for further model learning

In [4]:
!pip install sentence-transformers googletrans==3.1.0a0

In [6]:
from grammar_ru.corpus import ParallelCorpus, CorpusReader
from grammar_ru.common.separator.df_viewer import DfViewer
from grammar_ru.common.separator import Separator
from ca import Seq2VecMatcher
import pandas as pd
from pathlib import Path

### Data extracting

Extracting: books data, retells, data and their relations data from prepared previously parallel corpus.



In [ ]:
parallel_corpus = ParallelCorpus(Path('./files/parallel_corpus.zip'))

In [8]:
book_reader: CorpusReader = parallel_corpus.ru_book
book_firts_chapter_id = 'ru_book_0'
book_chapter_df = book_reader.get_frames(['ru_book_0']).first()

In [9]:
book_chapter_df

,word_id,sentence_id,word_index,paragraph_id,word_tail,word,word_type,word_length,file_id,corpus_id
0,0,0,0,0,1,В,ru,1,ru_book_0,parallel_corpus.zip
1,1,0,1,0,1,начале,ru,6,ru_book_0,parallel_corpus.zip
2,2,0,2,0,0,июля,ru,4,ru_book_0,parallel_corpus.zip
3,3,0,3,0,1,",",punct,1,ru_book_0,parallel_corpus.zip
4,4,0,4,0,1,в,ru,1,ru_book_0,parallel_corpus.zip
...,...,...,...,...,...,...,...,...,...,...
3238,3238,183,4,49,1,будто,ru,5,ru_book_0,parallel_corpus.zip
3239,3239,183,5,49,1,в,ru,1,ru_book_0,parallel_corpus.zip
3240,3240,183,6,49,1,некотором,ru,9,ru_book_0,parallel_corpus.zip
3241,3241,183,7,49,0,волнении,ru,8,ru_book_0,parallel_corpus.zip


Now we can use parallel corpus to get the first chapter of the retell by mapping.

In [10]:
retell_reader: CorpusReader = parallel_corpus.ru_retell
retell_reader.get_toc()
retell_firts_chapter_id = 'ru_retell_0'
retell_chapter_df = retell_reader.get_frames(['ru_retell_0']).first()
retell_chapter_df

,word_id,sentence_id,word_index,paragraph_id,word_tail,word,word_type,word_length,file_id,corpus_id,original_word_id,original_sentence_id,original_paragraph_id,updated
631302,631302,631302,0,631302,1,В,ru,1,ru_retell_0,parallel_corpus.zip,0,0,0,False
631303,631303,631302,1,631302,1,Петербурге,ru,10,ru_retell_0,parallel_corpus.zip,1,0,0,False
631304,631304,631302,2,631302,1,жаркое,ru,6,ru_retell_0,parallel_corpus.zip,2,0,0,False
631305,631305,631302,3,631302,1,и,ru,1,ru_retell_0,parallel_corpus.zip,3,0,0,False
631306,631306,631302,4,631302,1,душное,ru,6,ru_retell_0,parallel_corpus.zip,4,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
631693,631693,631326,16,631305,1,похожий,ru,7,ru_retell_0,parallel_corpus.zip,391,24,6,False
631694,631694,631326,17,631305,1,на,ru,2,ru_retell_0,parallel_corpus.zip,392,24,6,False
631695,631695,631326,18,631305,1,отставного,ru,10,ru_retell_0,parallel_corpus.zip,393,24,6,False
631696,631696,631326,19,631305,0,чиновника,ru,9,ru_retell_0,parallel_corpus.zip,394,24,6,False


In [11]:
retell_chapter_df = parallel_corpus.get_mapped_data([book_firts_chapter_id],['ru_book_ru_retell']).first()['ru_book_ru_retell']

In [12]:
retell_chapter_df

,word_id,sentence_id,word_index,paragraph_id,word_tail,word,word_type,word_length,file_id,corpus_id,original_word_id,original_sentence_id,original_paragraph_id,updated
631302,631302,631302,0,631302,1,В,ru,1,ru_retell_0,parallel_corpus.zip,0,0,0,False
631303,631303,631302,1,631302,1,Петербурге,ru,10,ru_retell_0,parallel_corpus.zip,1,0,0,False
631304,631304,631302,2,631302,1,жаркое,ru,6,ru_retell_0,parallel_corpus.zip,2,0,0,False
631305,631305,631302,3,631302,1,и,ru,1,ru_retell_0,parallel_corpus.zip,3,0,0,False
631306,631306,631302,4,631302,1,душное,ru,6,ru_retell_0,parallel_corpus.zip,4,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
631693,631693,631326,16,631305,1,похожий,ru,7,ru_retell_0,parallel_corpus.zip,391,24,6,False
631694,631694,631326,17,631305,1,на,ru,2,ru_retell_0,parallel_corpus.zip,392,24,6,False
631695,631695,631326,18,631305,1,отставного,ru,10,ru_retell_0,parallel_corpus.zip,393,24,6,False
631696,631696,631326,19,631305,0,чиновника,ru,9,ru_retell_0,parallel_corpus.zip,394,24,6,False


### Finally, we can see which sentences from the book and retell match.

For this task we will use the Seq2VecConverter class, get_matches method.

In [14]:
matcher = Seq2VecMatcher()
matched_ids, matched_sentences = matcher.get_matches(book_chapter_df,retell_chapter_df,need_matching_df=True)

In [15]:
matched_sentences

В начале июля, в чрезвычайно жаркое время, под вечер, один молодой человек вышел из своей каморки, которую нанимал от жильцов в С—м переулке, на улицу и медленно, как бы в нерешимости, отправился к К—ну мосту.1                                                                                   В Петербурге жаркое и душное лето. 
Он благополучно избегнул встречи с своею хозяйкой на лестнице.                                                                                                                                                                                                                         Один молодой человек выходит из своей каморки,...
Каморка его приходилась под самою кровлей высокого пятиэтажного дома и походила более на шкаф, чем на квартиру.                                                                                                                                                                        Один молодой человек выходит из своей каморки,...
Квартирная же

In [22]:
matched_sentences.array

<PandasArray>
[                                                                                                  'В Петербурге жаркое и душное лето. ',
             'Один молодой человек выходит из своей каморки, пытаясь избежать встречи с хозяйкой — он давно уже не платит за квартиру. ',
             'Один молодой человек выходит из своей каморки, пытаясь избежать встречи с хозяйкой — он давно уже не платит за квартиру. ',
             'Один молодой человек выходит из своей каморки, пытаясь избежать встречи с хозяйкой — он давно уже не платит за квартиру. ',
             'Один молодой человек выходит из своей каморки, пытаясь избежать встречи с хозяйкой — он давно уже не платит за квартиру. ',
             'Один молодой человек выходит из своей каморки, пытаясь избежать встречи с хозяйкой — он давно уже не платит за квартиру. ',
                                                    'Его гордость страдает от этого, а потому настроение героя нельзя назвать хорошим. ',
 'В последнее время 

In [23]:
matched_ids['MatchedWithSentence'] = matched_sentences.array

In [24]:
matched_ids

,sentence_id,MatchedWith,MatchedWithSentence
0,0,631302,В Петербурге жаркое и душное лето.
1,1,631303,"Один молодой человек выходит из своей каморки,..."
2,2,631303,"Один молодой человек выходит из своей каморки,..."
3,3,631303,"Один молодой человек выходит из своей каморки,..."
4,4,631303,"Один молодой человек выходит из своей каморки,..."
...,...,...,...
179,179,631325,Он набрел на трактир и вошел.
180,180,631326,"Там было мало народу, но Раскольникову бросилс..."
181,181,631326,"Там было мало народу, но Раскольникову бросилс..."
182,182,631326,"Там было мало народу, но Раскольникову бросилс..."


### Now we can see the matched sentences.

In [25]:
df_to_display = pd.merge(book_chapter_df,matched_ids,left_on='sentence_id',right_on='sentence_id',how='left')
sentences_to_display = matcher.viewer.to_sentences_strings(df_to_display,'MatchedWith').to_frame()
sentences_to_display = pd.merge(df_to_display,sentences_to_display,left_on='MatchedWith',right_on='MatchedWith',how='left')

In [26]:
sentences_to_display

,word_id,sentence_id,word_index,paragraph_id,word_tail,word,word_type,word_length,file_id,corpus_id,MatchedWith,MatchedWithSentence,word_print
0,0,0,0,0,1,В,ru,1,ru_book_0,parallel_corpus.zip,631302,В Петербурге жаркое и душное лето.,"В начале июля, в чрезвычайно жаркое время, под..."
1,1,0,1,0,1,начале,ru,6,ru_book_0,parallel_corpus.zip,631302,В Петербурге жаркое и душное лето.,"В начале июля, в чрезвычайно жаркое время, под..."
2,2,0,2,0,0,июля,ru,4,ru_book_0,parallel_corpus.zip,631302,В Петербурге жаркое и душное лето.,"В начале июля, в чрезвычайно жаркое время, под..."
3,3,0,3,0,1,",",punct,1,ru_book_0,parallel_corpus.zip,631302,В Петербурге жаркое и душное лето.,"В начале июля, в чрезвычайно жаркое время, под..."
4,4,0,4,0,1,в,ru,1,ru_book_0,parallel_corpus.zip,631302,В Петербурге жаркое и душное лето.,"В начале июля, в чрезвычайно жаркое время, под..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3238,3238,183,4,49,1,будто,ru,5,ru_book_0,parallel_corpus.zip,631326,"Там было мало народу, но Раскольникову бросилс...",Но никто не разделял его счастия; молчаливый т...
3239,3239,183,5,49,1,в,ru,1,ru_book_0,parallel_corpus.zip,631326,"Там было мало народу, но Раскольникову бросилс...",Но никто не разделял его счастия; молчаливый т...
3240,3240,183,6,49,1,некотором,ru,9,ru_book_0,parallel_corpus.zip,631326,"Там было мало народу, но Раскольникову бросилс...",Но никто не разделял его счастия; молчаливый т...
3241,3241,183,7,49,0,волнении,ru,8,ru_book_0,parallel_corpus.zip,631326,"Там было мало народу, но Раскольникову бросилс...",Но никто не разделял его счастия; молчаливый т...


In [27]:
## доделать вместо word_print возникало предложение пересказ
DfViewer().highlight('word_print','auto').tooltip('MatchedWithSentence').to_html_display(sentences_to_display)

In [29]:
Separator.Viewer().to_sentences_strings(sentences_to_display)

sentence_id
0      В начале июля, в чрезвычайно жаркое время, под...
1      Он благополучно избегнул встречи с своею хозяй...
2      Каморка его приходилась под самою кровлей высо...
3      Квартирная же хозяйка его, у которой он нанима...
4      И каждый раз молодой человек, проходя мимо, чу...
                             ...                        
179                                 Свою прежнюю нашел…4
180    Но никто не разделял его счастия; молчаливый т...
181    Был тут и еще один человек, с виду похожий как...
182    Он сидел особо, перед своею посудинкой, изредк...
183          Он был тоже как будто в некотором волнении.
Name: word_print, Length: 184, dtype: object

In [30]:
sent_dataset = sentences_to_display[['MatchedWithSentence', 'word_print']]
sent_dataset = sent_dataset.drop_duplicates(subset=['word_print'], keep='first', )
sent_dataset = sent_dataset.reset_index(drop=True)
sent_dataset

,MatchedWithSentence,word_print
0,В Петербурге жаркое и душное лето.,"В начале июля, в чрезвычайно жаркое время, под..."
1,"Один молодой человек выходит из своей каморки,...",Он благополучно избегнул встречи с своею хозяй...
2,"Его гордость страдает от этого, а потому настр...","Не то чтоб он был так труслив и забит, совсем ..."
3,В последнее время он вообще стал напоминать ип...,Он до того углубился в себя и уединился от все...
4,"Выйдя на улицу, он обдумывает какое-то таинств...",Насущными делами своими он совсем перестал и н...
5,"Его тревожит, что он не соответствует тому дел...","Никакой хозяйки, в сущности, он не боялся, что..."
6,"Герой нарочно растравляет себя мыслями о том, ...","Но останавливаться на лестнице, слушать всякий..."
7,Колорит улиц вторит его мрачным думам: везде п...,«На какое дело хочу покуситься и в то же время...
8,На улице его грубо окликнули «немецким шляпник...,"Любопытно, чего люди больше всего боятся?"
9,"Он рассуждает про себя, что шляпа погубит все ...","Нового шага, нового собственного слова они все..."


In [31]:
sent_dataset_json = sent_dataset.to_json(orient="records", force_ascii=False)

In [34]:
sent_dataset_json
print()

KeyboardInterrupt: 

#### Write data from matcher to .json file.

In [33]:
file_path = "data/chapter_0.json"

import os
os.makedirs(os.path.dirname(file_path), exist_ok=True)

with open(file_path, 'w', encoding='utf-8') as file:
    file.write(sent_dataset_json)